# --- Day 9: Rope Bridge ---

For more detail, see https://adventofcode.com/2022/day/9 

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

## My approach

Represent the positions of the head (H) and tail (T) relative to the starting position ([0 0]) as numpy arrays containing x,y coordinates.

Each step in the series of motions will result in a displacement (D) of the head, horizontally or vertically, by one unit in the given direction.

The displacement of the head will result in tension in the rope. The force (F) of this tension is proportional to the Euclidean distance. To avoid dealing with rounding errors, we'll work with the square of the distance.

If head and tail are adjacent, including diagonally, Fsq <= 2 and the tail doesn't move.

If the head moves to a position 2 units horizontally or vertically from the tail, now Fsq > 2 and the tail moves one unit in the direction of the nonzero F component. The tail displacement is thus sign(F).

If the head is already diagonally adjacent, and it moves farther from the tail, then the tail will move one unit in each dimension toward the head. The tail displacement in this case is still sign(F).

If the tail is in a location we haven't visited yet, make a note of it. Store the coordinates in a dictionary as a text label. A list would also work, but I thought the dictionary might offer more flexibility for the unknown part 2.

In [1]:
import numpy as np

with open('ropemoves.txt', 'r') as f:
    moves = f.read().split('\n')

H = np.array([0, 0])
T = np.array([0, 0])
visited = {}
visited[np.array2string(T)] = 1

for line in moves:
    direc = line.split()[0]
    steps = int(line.split()[1])

    for i in range(steps):
        if direc=="R":
            D = np.array([1, 0])
        if direc=="U":
            D = np.array([0, 1])
        if direc=="L":
            D = np.array([-1, 0])
        if direc=="D":
            D = np.array([0, -1])
        H = H + D

        F = H - T
        Fsq = F[0]**2 + F[1]**2

        if Fsq > 2:
            T = T + np.sign(F)
            if not np.array2string(T) in visited:
                visited[np.array2string(T)] = 1

len(visited)

6236

## --- Part Two ---

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?

Be careful: more types of motion are possible than before.

### My approach
Since knots other than the head can move diagonally, the rope can now stretch up to 2 diagonal positions between knots. The force is a little stronger but the previous logic still holds: the next knot will move one unit in each dimension towards the knot it's following so its displacement is sign(F).

Using a 2D array, we can represent a rope with an arbitrary number of knots. 

In [2]:
knots = 10
rope = np.array([[0, 0] for i in range(knots)])

visited = {}
visited[np.array2string(rope[knots-1])] = 1

for line in moves:
    direc = line.split()[0]
    steps = int(line.split()[1])

    for i in range(steps):
        if direc=="R":
            D = np.array([1, 0])
        if direc=="U":
            D = + np.array([0, 1])
        if direc=="L":
            D = + np.array([-1, 0])
        if direc=="D":
            D = + np.array([0, -1])
        rope[0] = rope[0] + D

        for k in range(1, knots):          
            F = rope[k-1] - rope[k]
            Fsq = F[0]**2 + F[1]**2
            if Fsq > 2:
                rope[k] += np.sign(F)
            if k==knots-1 and not np.array2string(rope[k]) in visited:
                visited[np.array2string(rope[k])] = 1
                    

In [3]:
len(visited)

2449